# 1. web data
 we’d want at least a 5-year history which will involve pulling from 3 different sites from what I’ve seen.

 Eventually we’ll want every district’s data so we can compare to the entire state average. I made a ranking in column F if there is a need to prioritize, Dallas ISD is the second most important district. One thing I noticed is that my list uses the whole phrase “Independent School District”, but the website uses the abbreviation of “ISD” they’re the same thing.

 

For the file format, we’re going to have all the data loaded into an internal database and my guess would be that it’d be easiest to do that if they were all in the same xlsx but I’m not an expert on the best way to structure that, so I’ll defer to Che’s experiences. For each district it would be helpful to have their district ID in the file, for example Dallas’s ID is 057905.

## site 1. 
The first site has the 3 most recent school years and should be where we start, for our example of Dallas ISD you’ll need to select the below inputs and Search then View the Report. Once you view the report, you’ll have to use the “Prev Year” link to get the older data.

School years: 2021-22, 2020-21, and 2018-19
 

#### 1. get district id:
   
https://schoolsdata2-93b5c-tea-texas.opendata.arcgis.com/datasets/districts2020to2021/explore?location=32.706217%2C-96.751111%2C10.31

https://schoolsdata2-db440-tea-texas.opendata.arcgis.com/ 

match id with sample file -> check missing district id.

In [ ]:


k12=pd.read_csv('Capstone Project_ Impact Primary_Secondary Education/K_12_HIPinvestor.csv')
k12_tx=k12[k12['State Abbreviation']=='TX']
Ktx_dname=list(k12_tx['Impact Entity Name'].str.replace(', TX','').str.lower())
print(len(Ktx_dname))
#print(len(k12_tx['Impact Entity Name'].str.replace(', TX','').str.lower())) # 1210

# Check duplicate district name
from collections import Counter
d_dup1=[k for k, v in Counter(Ktx_dname).items() if v > 1]
print(d_dup1)

Ktx_dname=set(Ktx_dname)
Ktx=pd.DataFrame(Ktx_dname,columns=['district_name'])
Ktx

1210
['big sandy isd', 'centerville isd', 'chapel hill isd', 'dawson isd', 'edgewood isd', 'highland park isd', 'hubbard isd', 'midway isd', 'northside isd', 'valley view isd', 'wylie isd']


,district_name
0,hemphill isd
1,sabine isd
2,san marcos cisd
3,crawford isd
4,albany isd
...,...
1194,rocksprings isd
1195,guthrie csd
1196,onalaska isd
1197,clarksville isd


In [ ]:
# Reference: https://schoolsdata2-93b5c-tea-texas.opendata.arcgis.com/datasets/school2020to2021/explore?location=31.071573%2C-100.146608%2C6.75
district=pd.read_csv('TXdistrict/School2020to2021.csv')[['District_N','District_1']].rename(columns={'District_1':'district_id','District_N':'district_name'})
district['district_name']=district['district_name'].str.lower()
district['district_id']=district['district_id'].apply(lambda x: str(x).zfill(6))
district=district.drop_duplicates()
district

,district_name,district_id
0,weslaco isd,108913
1,donna isd,108902
2,wylie isd,043914
3,round rock isd,246909
4,college station isd,021901
...,...,...
9460,yantis isd,250905
9465,yellowstone college preparatory,101873
9482,yoakum isd,062903
9488,yorktown isd,062904


In [ ]:
Ktx=pd.merge(Ktx,district,how='left',on='district_name')  # The number match
Ktx 

,district_name,district_id
0,hemphill isd,202903
1,sabine isd,092906
2,san marcos cisd,105902
3,crawford isd,161901
4,albany isd,209901
...,...,...
1205,rocksprings isd,069901
1206,guthrie csd,135001
1207,onalaska isd,187910
1208,clarksville isd,194904


In [ ]:
# Check duplicate district name
from collections import Counter
d_dup2=[k for k, v in Counter(district['district_name']).items() if v > 1]
print(d_dup2)

['wylie isd', 'northside isd', 'edgewood isd', 'highland park isd', 'big sandy isd', 'midway isd', 'centerville isd', 'chapel hill isd', 'dawson isd', 'hubbard isd', 'valley view isd']


In [ ]:
set(d_dup1)==set(d_dup2)

True

**Conclusion: All school distribute numbers in Texas have been found.**

Save in dataframe Ktx

#### 2.1 web scraping for 3 year: 2021-22, 2020-21, and 2018-19 -> get excel report directly


**Reference Link**

base='https://rptsvr1.tea.texas.gov/perfreport/tprs/tprs_srch.html'

https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=0&lev=D&id=057905&prgopt=reports%2Ftapr%2Fperformance.sas

https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=0&lev=D&id=061912&prgopt=reports%2Ftapr%2Fperformance.sas

Example:  id='057905', year='2021' 

ureport='https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=0&lev=D&id='+id+'&prgopt=reports%2Ftapr%2Fperformance.sas'

this link will download a excel directly
'https://rptsvr1.tea.texas.gov//cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=0&lev=D&id=057905&prgopt=reports%2Ftapr%2Fperformance.sas&dest=E'
uexcel='https://rptsvr1.tea.texas.gov//cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=0&lev=D&id='+id+'&prgopt=reports%2Ftapr%2Fperformance.sas&dest=E'

21-22:
https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=0&lev=D&id=057905&prgopt=reports/tapr/performance.sas&ccyy_value=2022

20-21:
https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=0&lev=D&id=057905&prgopt=reports/tapr/performance.sas&ccyy_value=2021

18-19:
https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=0&lev=D&id=057905&prgopt=reports/tapr/performance.sas&ccyy_value=2019



In [ ]:
import os


years=['2019','2021','2022']
for name,id in zip(Ktx['district_name'],Ktx['district_id']):
    for year in years:
        uexcel='https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=0&lev=D&id={}&prgopt=reports/tapr/performance.sas&ccyy_value={}&dest=E'.format(id,year)
        response = requests.get(uexcel)
        
        folder_name = name
        folder = os.path.join('excel report', folder_name)
        file_name = id+'_'+year+'.xlsx'
        file = os.path.join(folder, file_name)
        os.makedirs(folder,exist_ok=True)
        with open(file, 'wb') as f:
            f.write(response.content) 
            
            